## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Bredasdorp,-34.5322,20.0403,66.99,82,5,1.01,ZA,clear sky
1,1,Ribeira+Grande,38.5167,-28.7000,64.40,88,40,16.11,PT,scattered clouds
2,2,Ushuaia,-54.8000,-68.3000,53.60,54,20,25.32,AR,few clouds
3,3,Falmouth,50.1544,-5.0711,45.00,93,90,24.16,GB,light rain
4,4,Atar,20.5169,-13.0499,72.59,18,7,9.60,MR,clear sky


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                289
City                   289
Lat                    289
Lng                    289
Max Temp               289
Humidity               289
Cloudiness             289
Wind Speed             289
Country                283
Current Description    289
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                283
City                   283
Lat                    283
Lng                    283
Max Temp               283
Humidity               283
Cloudiness             283
Wind Speed             283
Country                283
Current Description    283
dtype: int64

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atar,MR,72.59,clear sky,20.5169,-13.0499,
7,Cape+Town,ZA,72.00,clear sky,-33.9258,18.4232,
8,Nouadhibou,MR,75.20,broken clouds,20.9310,-17.0347,
11,Rocha,UY,75.47,moderate rain,-34.4833,-54.3333,
12,Saint-Philippe,RE,78.80,clear sky,-21.3585,55.7679,
13,Ouricuri,BR,78.62,overcast clouds,-7.8825,-40.0817,
18,Mana,GF,78.80,overcast clouds,5.6592,-53.7767,
19,Luba,GQ,80.60,few clouds,3.4568,8.5547,
20,Beloha,MG,77.32,clear sky,-25.1667,45.0500,
23,Sao+Filipe,CV,75.79,overcast clouds,14.8961,-24.4956,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
#     print(json.dumps(hotels,indent = 4))
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... skipping")

Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not found.... skipping
Hotel not foun

In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
# hotel_df = hotel_df.dropna()
# hotel_df.head()

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

In [26]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))